## 官方的代码

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00


### 提取数据

下面代码就是基于上一节学的内容，整个的原理就是先选取一个预先训练好的模型，他懂得基本的语言，然后……

就需要一个叫做 tokenizer（分词器） 的好帮手。

它的作用，就像一个翻译官，负责把我们人类能看懂的句子（比如 "I love this course!"），翻译成模型能“吃”进去的数字格式。没有这个翻译过程，模型面对我们的文字就像是在看天书。

具体来说，tokenizer 在这里主要做了几件重要的事：

分词 (Tokenization): 把一整句话，按照模型能理解的方式，切成一个个有意义的小单元（这些小单元叫做 token）。比如 "waiting" 可能会被切成 ["wait", "##ing"]。

转为ID (Convert to IDs): tokenizer 心里有一本厚厚的“字典”（专业上叫 vocabulary），里面给每一个 token 都编了一个独一无二的数字ID。这一步就是把切好的 token 们，按照字典查表，转换成一串数字。

整理队形 (Padding & Truncation):

当我们一次性给模型喂好几句话（一个 batch）的时候，模型要求这些话的长度必须完全一样。

padding=True 的作用就是，自动找出这批句子里最长的那一句，然后用一个特殊的、表示“占位”的 [PAD] 符号，把其他短的句子都补到和最长的一样长。

truncation=True 则是反过来，如果有一句话太长了，超过了模型能处理的最大限度，它就会自动把超出的部分砍掉。

生成“注意力小抄” (Attention Mask):

因为我们用 [PAD] 符号补齐了句子，但这些是“假”的词，我们不希望模型去关注它们。

所以 tokenizer 会非常贴心地同时生成一个叫做 attention_mask 的东西。它是一串由 1 和 0 组成的列表，1 的位置对应的是真实的单词，0 的位置对应的就是补充的 [PAD]。这样模型拿到手就知道该把注意力放在哪里了。

指定输出格式 (Return Tensors):

return_tensors="pt" 的意思是，把上面所有处理好的数字，都打包成 PyTorch 能直接使用的张量（Tensor）格式。

所以，batch = tokenizer(...) 这行代码执行完后，我们得到的 batch 就不再是简单的句子了，而是一个包含了**input_ids（就是上面说的数字ID）、attention_mask**（“注意力小抄”）等信息的、可以直接喂给模型的“数据大礼包”。

最后，我们手动给这个“大礼包”加上 batch["labels"] = torch.tensor([1, 1])，也就是为这批数据贴上“正确答案”。

这样，当模型吃掉这个带有正确答案的 batch 后 (model(**batch))，它就能自己算出 loss（表示模型预测的结果和“正确答案”之间的差距）。我们再通过 loss.backward() 和 optimizer.step() 这两步，就能让模型根据这次的差距，进行一次小小的学习和自我修正了。

In [2]:
import torch
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 和之前一样
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# 新增部分
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [2]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [3]:
raw_train_dataset.features

{'sentence1': Value('string'),
 'sentence2': Value('string'),
 'label': ClassLabel(names=['not_equivalent', 'equivalent']),
 'idx': Value('int32')}

In [4]:
# 训练集的第 15 行元素和验证集的 87 行元素
raw_train_dataset[15], raw_datasets["validation"][87]

({'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .',
  'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .',
  'label': 0,
  'idx': 16},
 {'sentence1': 'However , EPA officials would not confirm the 20 percent figure .',
  'sentence2': 'Only in the past few weeks have officials settled on the 20 percent figure .',
  'label': 0,
  'idx': 812})

### 预处理数据

In [13]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"][:])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"][:])

'''

这里源代码是
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

'''

'\n\n这里源代码是\ntokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])\ntokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])\n\n'

关于 tokenizer(raw_datasets["train"]["sentence1"]) 这行代码，需要特别注意。

原版 raw_datasets["train"]["sentence1"] 之所以会报错，是因为 Hugging Face 的 datasets 库为了节省内存，做了一个非常聪明的优化。当咱们像这样取一整列数据时，它并不会立刻把成千上万条句子全部加载到内存里，变成一个巨大的Python列表（List）。

相反，它返回的是一个特殊的、轻量级的“数据集列对象”（Dataset Column Object）。你可以把它想象成一张“菜谱清单”，它只告诉你“仓库里有什么食材”，但并没有把食材真的都搬出来。

然而，我们的 tokenizer (翻译官/厨师) 并不知道该如何处理这张“清单”，他需要的是实实在在的“食材”（也就是真实的句子列表）才能开始工作。

而当我们加上 [:] 这个小小的切片操作后，raw_datasets["train"]["sentence1"][:] 就等于下达了一个明确的指令：“现在，立刻去仓库，把这张清单上的所有食材（全部句子）都给我拿出来，放进一个篮子里！”

这个操作会强制将那个特殊对象里的所有数据都读取到内存中，并转换成一个 tokenizer 能看懂的标准Python列表。这样，代码就能成功运行了。

但是这么做有一个弊端，就是全部处理了之后会非常的占用内存，因此只适用于小型的数据集。

别忘了tokenizer生成的是，如果这个位置有单词就是1，填充物就是0。

In [10]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

input_ids指的是句子的内容是什么
token_type_ids指的是内容属于哪个句子
attention_mask指的是哪些内容需要关注

In [18]:
# 从数据集中，分别取出第15行的两个原始句子（它们都是字符串）
sentence_1_text = raw_datasets["train"]["sentence1"][15]
sentence_2_text = raw_datasets["train"]["sentence2"][15]

# 打印出来看看，确认它们是纯文本
print("第一句话原文:", sentence_1_text)
print("第二句话原文:", sentence_2_text)
print("-" * 20)

# 直接把两个句子字符串作为参数，一次性喂给 tokenizer！
# 它会自动把它们拼接到一起，并生成正确的 token_type_ids
inputs = tokenizer(sentence_1_text, sentence_2_text)

# 现在，inputs 里面就是我们想要的、包含 'input_ids', 'token_type_ids', 'attention_mask' 的正确结果啦！
print("正确处理后的结果:")
print(inputs)


第一句话原文: Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .
第二句话原文: Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .
--------------------
正确处理后的结果:
{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [19]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'rudder',
 'was',
 'most',
 'recently',
 'senior',
 'vice',
 'president',
 'for',
 'the',
 'developer',
 '&',
 'platform',
 'evan',
 '##gel',
 '##ism',
 'business',
 '.',
 '[SEP]',
 'senior',
 'vice',
 'president',
 'eric',
 'rudder',
 ',',
 'formerly',
 'head',
 'of',
 'the',
 'developer',
 'and',
 'platform',
 'evan',
 '##gel',
 '##ism',
 'unit',
 ',',
 'will',
 'lead',
 'the',
 'new',
 'entity',
 '.',
 '[SEP]']

In [21]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"][:],
    raw_datasets["train"]["sentence2"][:],
    padding=True,
    truncation=True,
)

In [22]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [23]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

### 动态填充

In [24]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# 这个函数自动padding，下面的只是演示

In [25]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

In [26]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

## 在 GLUE SST-2 数据集上复刻上述预处理

In [28]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "sst2")

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

## 对所有通用的glue进行处理

In [29]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# "mrpc", "sst2", "qnli", "cola"
task = "qnli"
# =======================================================

checkpoint = "bert-base-uncased"

# --- 第一步：加载数据集和分词器 ---
# 这里的一切都和之前一样，只是任务名变成了一个变量
print(f"正在为GLUE任务 '{task}' 加载数据...")
raw_datasets = load_dataset("glue", task)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


# --- 第二步：创建“魔法对照表”，告诉代码每个任务的句子列名叫什么 ---
# 这就是我们“万能工具”的核心秘密！
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}


# --- 第三步：根据当前任务，从“对照表”里取出正确的列名 ---
sentence1_key, sentence2_key = task_to_keys[task]
print(f"识别到任务 '{task}' 的输入列为: '{sentence1_key}'" + (f" 和 '{sentence2_key}'" if sentence2_key else ""))


# --- 第四步：定义万能处理函数 ---
def universal_tokenize_function(examples):
    # 如果对照表里第二列是None，说明这是个单句子任务
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    # 否则，就是个双句子任务
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)


# --- 第五步：启动我们的“万能生产线”！ ---
# .map() 会把我们的通用函数应用到数据集的每一行
print("正在启动通用处理流水线...")
tokenized_datasets = raw_datasets.map(universal_tokenize_function, batched=True)
print("数据预处理完成！")


# --- 第六步（可选）：检查一下，并准备好我们的“配餐大师” ---
# 这一步和之前完全一样，我们的“配餐大师”依然胜任！
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 现在，无论你一开始选择哪个GLUE任务，
# tokenized_datasets 都已经是被完美处理好的状态，
# data_collator 也已经准备好为后续的DataLoader服务啦！


正在为GLUE任务 'qnli' 加载数据...


qnli/train-00000-of-00001.parquet:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

qnli/validation-00000-of-00001.parquet:   0%|          | 0.00/872k [00:00<?, ?B/s]

qnli/test-00000-of-00001.parquet:   0%|          | 0.00/877k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

...

识别到任务 'qnli' 的输入列为: 'question' 和 'sentence'
正在启动通用处理流水线...


Map:   0%|          | 0/104743 [00:00<?, ? examples/s]

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

数据预处理完成！
